<h3>Análise Inicial dos Dados</h3>
<h4>Variáveis Preditoras:</h4>
<pre>
Índice     Nome                 Tipo
.................................................
   1       COD_UNIDADE          Categórica-Nominal
   2       COD_TURMA            Categórica-Nominal
  12       DURACAO_CURSO        Numérica-Discreta
  17       NR_TOTAL_DISCIPLINAS Numérica-Discreta
  20       T_IDADE              Numérica-Discreta
  21       NOTA                 Numérica-Contínua
  23       POSSUI_ENEM          Numérica-Discreta 
  27       SEMESTRES_CURSADOS   Numérica-Discreta
  28       POSSUI FIES          Categórica-Ordinal
</pre>

<h4>Classes:</h4>
<pre>
Nome                 Tipo
......................................
SEMESTRES_ATRASADOS  Numérica-Discreta


Foram identificadas 29 classes: 
    [0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 
    15 16 18 20 21 22 24 40 57 62 64 80 90 98]
</pre>

In [1]:
# importar bibliotecas python
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

C:\Users\ebteles\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# importar dados
base = pd.read_csv('case_Formatura.csv', encoding='ISO-8859-1')

In [3]:
# Definir e tratar variáveis preditoras
previsores = base.iloc[:,[1, 2, 12, 17, 20, 21, 23, 27, 28]]

# Identificar e remover alunos não formados
total_nao_formado = previsores.loc[previsores['SEMESTRES_CURSADOS'] < previsores['DURACAO_CURSO']].shape[0]

if total_nao_formado > 0:
    previsores.drop(previsores.loc[previsores['SEMESTRES_CURSADOS'] < previsores['DURACAO_CURSO']].index, inplace=True)
    print('Foram removidos {} registros da base'.format(total_nao_formado))
else:
    print('Nenhum registro foi removido')

Foram removidos 36 registros da base


C:\Users\ebteles\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
labelencoder_previsores = LabelEncoder()
previsores.iloc[:, 0] = labelencoder_previsores.fit_transform(previsores.iloc[:, 0]) # unidade
previsores.iloc[:, 1] = labelencoder_previsores.fit_transform(previsores.iloc[:, 1]) # turma
previsores.iloc[:, 8] = labelencoder_previsores.fit_transform(previsores.iloc[:, 8]) # possui fies

C:\Users\ebteles\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
# Definir e tratar classes (semestres para conclusão do curso)
classe = pd.DataFrame(previsores['SEMESTRES_CURSADOS'] - previsores['DURACAO_CURSO'], columns=['SEMESTRES_ATRASADOS'])
total_classe = classe['SEMESTRES_ATRASADOS'].unique().shape[0] 
# print(classes.describe())
# print('Total de classes: ', total_classe)
# print('Lista de classes: ', np.sort(classe['SEMESTRES_ATRASADOS'].unique()))

In [13]:
# escalonamento
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [21]:
# segmentação: treino e teste
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.10, random_state=0)

In [22]:
# importação da biblioteca
classificador = LogisticRegression()
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

C:\Users\ebteles\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
# criação do classificador
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

C:\Users\ebteles\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
# matriz de confusao
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)

In [25]:
import collections
collections.Counter(classe_teste)

Counter({'SEMESTRES_ATRASADOS': 1})

In [26]:
print('Precisão =', precisao)
print('Fim')

Precisão = 0.846053205408
Fim
